In [2]:
import time
import share
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import os
import urllib.request as req

In [2]:
data_path = './data'
out_path = './output'

# 웹 크롤링(Selenium)

In [3]:
options = webdriver.ChromeOptions()
# 작업 후에도 브라우저 창을 열어두도록 설정
options.add_experimental_option('detach',True)
driver = webdriver.Chrome(options=options)

## 1 다나와 로그인 후 관심 상품 가져오기

### 다나와 로그인 화면 접속

In [4]:
url = 'https://www.danawa.com/'

driver.get(url) # 해당 주소로 이동

# driver.implicitly_wait(second) : 페이지가 로딩될 때 second초 만큼 기다린다.
driver.implicitly_wait(3)

driver.maximize_window() # 창을 최대화

# 사이트에 따라 너무 빨리 동작을 하면 크롤링으로 판정할 수 있으므로 일정 초수로 쉬어주는 것이 좋다.
# 상이트에 따라 일정 시간을 규칙적으로 쉬면 크롤링으로 판정하는 경우도 있다.
time.sleep(3) #3초 기다린다.

login = driver.find_element(By.CSS_SELECTOR,'#danawa_header > div > div > div.main-header__banner > div.main-header__user > div:nth-child(5) > a' ) # css selector를 이용해서 요소 선택

print(f'HTML 요소: {login}')
print(f'태그 이름: {login.tag_name}')
print(f'문자열: {login.text}')
print(f'href 요소: {login.get_attribute("href")}')

login.click() # 마우스 클릭 동작 구현
driver.implicitly_wait(3)
time.sleep(3)

HTML 요소: <selenium.webdriver.remote.webelement.WebElement (session="3cacfe69bbe5ce47a72424a005a693d8", element="f.19EA27B5744836A072F113A3D302DE4B.d.0B178F98ED121C311A5B7FFA9AACA2E3.e.466")>
태그 이름: a
문자열: 로그인
href 요소: https://auth.danawa.com/login?url=https%3A%2F%2Fwww.danawa.com%2F


### 다나와 로그인 실행

In [5]:
my_id, my_pass = share.get_danawa_login_info()
# send_keys() : 해당 요소에 키 입력
# send_keys(Keys.ENTER)-> 엔터키를 입력
driver.find_element(By.ID, 'danawa-member-login-input-id').send_keys(my_id) # 아이디 입력
time.sleep(2)
driver.find_element(By.NAME, 'password').send_keys(my_pass) # 비밀번호 입력
driver.find_element(By.CSS_SELECTOR, '#danawa-member-login-loginButton').click() # 버튼 클릭
time.sleep(2)

### 다나와 관심상품 가져오기

In [6]:
# 관심 상품 목록 HTML 펴이지 가져오기
heart = driver.find_element(By.CSS_SELECTOR,'#danawa_header > div > div > div.main-header__banner > div.main-header__user > div:nth-child(4) > a')
heart.click() # 마우스 클릭 동작 구현
driver.implicitly_wait(3)
html = driver.page_source
# 뷰티풀숲으로 파싱하기
soup = bs(html,'html.parser')
tag = soup.select('#wishProductListArea > table > tbody')[0]
rows = tag.select('tbody tr')

for item in rows:
    title = item.find('div', {'class':'tit'}).text
    price = item.find('span',{'class':'price'}).text.strip()
    link = item.find('a').get('href')
    print('-'*50)
    print(f'관심 품목 : {title}')
    print(f'관심 품목 가격 : {price}')
    print(f'링크 : {link}')

--------------------------------------------------
관심 품목 : APPLE 워치 SE 2세대 40mm 실버 알루미늄 (스포츠밴드)
관심 품목 가격 : 312,550원
링크 : https://prod.danawa.com/info/?pcode=13156116
--------------------------------------------------
관심 품목 : Microsoft 서피스 프로11 X Elite (램16GB, SSD 512GB)
관심 품목 가격 : 1,997,090원
링크 : https://prod.danawa.com/info/?pcode=54314645
--------------------------------------------------
관심 품목 : 로지텍 LIFT VERTICAL (정품) (그라파이트)
관심 품목 가격 : 71,000원
링크 : https://prod.danawa.com/info/?pcode=16909604
--------------------------------------------------
관심 품목 : 신일전자 air S10 SIF-E9CJB (1개)
관심 품목 가격 : 141,540원
링크 : https://prod.danawa.com/info/?pcode=52156871


## 2 유튜브 검색 결과 가져오기

In [11]:
options = webdriver.ChromeOptions()
options.add_experimental_option('detach',True)
# 드라이버 실행
driver = webdriver.Chrome(options=options)

In [4]:
keyword = '푸바오'
url = f'https://www.youtube.com/results?search_query={keyword}'

driver.get(url) # 웹 브라우저를 실행해 지정한 url에 접속
time.sleep(3)

print(f'접속한 웹 사이트의 제목 : {driver.title}')
print(f'접속한 웹 사이트의 url: {driver.current_url}')

접속한 웹 사이트의 제목 : 푸바오 - YouTube
접속한 웹 사이트의 url: https://www.youtube.com/results?search_query=%ED%91%B8%EB%B0%94%EC%98%A4


### 정렬 기준 지정 해서 불러오기

In [5]:
# 유튜브의 경우 내부 알고리즘으로 정렬을 하기 때문에
# 원하는 결과와 다를 수 있다. 검색 시 일정한 값이 나오지 않는다.

# 필터를 이용해서 정리
# 관련성 : https://www.youtube.com/results?search_query=%ED%91%B8%EB%B0%94%EC%98%A4&sp=CAASAhAB
# 업로드 날짜 : https://www.youtube.com/results?search_query=%ED%91%B8%EB%B0%94%EC%98%A4&sp=CAI%253D
# 조회수 : https://www.youtube.com/results?search_query=%ED%91%B8%EB%B0%94%EC%98%A4&sp=CAMSAhAB
# 평점 : https://www.youtube.com/results?search_query=%ED%91%B8%EB%B0%94%EC%98%A4&sp=CAESAhAB
# sp= 뒤의 문자열을 사용하면 정렬 기준을 적용해서 불러올 수 있다.

In [6]:
# 조회수 기준으로 불러온다.
keyword = '푸바오'
url = f'https://www.youtube.com/results?search_query={keyword}&sp=CAMSAhAB'

driver.get(url) # 웹 브라우저를 실행해 지정한 url에 접속
time.sleep(3)

print(f'접속한 웹 사이트의 제목 : {driver.title}')
print(f'접속한 웹 사이트의 url: {driver.current_url}')

접속한 웹 사이트의 제목 : 
접속한 웹 사이트의 url: https://www.youtube.com/results?search_query=%ED%91%B8%EB%B0%94%EC%98%A4&sp=CAMSAhAB


### 영상 제목, 링크 추출

In [7]:
html = driver.page_source # request get 함수와 동일
soup = bs(html,'html.parser')

# id값으로 video-title을 사용하는 a태그 모두 가져오기
titles = soup.select('a#video-title')

# # <a id="video-title" class="yt-simple-endpoint style-scope ytd-video-renderer" title="하부지랑 같이 놀고 싶었던 찰거머리 아기판다 | 에버랜드 판다월드 푸바오 (Baby Panda 'FuBao')" aria-label="하부지랑 같이 놀고 싶었던 찰거머리 아기판다 | 에버랜드 판다월드 푸바오 (Baby Panda 'FuBao') 게시자: 에버랜드 - EVERLAND 조회수 16,304,890회 3년 전 1분 56초" href="/watch?v=zH9n4777x80&amp;pp=ygUJ7ZG467CU7Jik">
#             <yt-icon id="inline-title-icon" class="style-scope ytd-video-renderer" hidden=""><!--css-build:shady--><!--css-build:shady--></yt-icon>
#             <yt-formatted-string class="style-scope ytd-video-renderer" aria-label="하부지랑 같이 놀고 싶었던 찰거머리 아기판다 | 에버랜드 판다월드 푸바오 (Baby Panda 'FuBao') 게시자: 에버랜드 - EVERLAND 조회수 16,304,890회 3년 전 1분 56초">하부지랑 같이 놀고 싶었던 찰거머리 아기판다 | 에버랜드 판다월드 푸바오 (Baby Panda 'FuBao')</yt-formatted-string>
#           </a>

# print(titles)

In [12]:
# 제목 출력
print(titles[0].text.strip())

판다 할배와 팔짱 데이트 | 에버랜드 아기판다 #푸바오 (Panda FuBao) #shorts


In [13]:
# 링크 주소
base_url = 'https://www.youtube.com'
print(base_url + titles[0]['href'])

https://www.youtube.com/shorts/IenW7-OrBVU


### 조회수, 업로드 시기

In [14]:
# <span class="inline-metadata-item style-scope ytd-video-meta-block">조회수 1630만회</span>
view_upload = soup.select('span.inline-metadata-item.style-scope.ytd-video-meta-block')
print((view_upload[:6]))

[<span class="inline-metadata-item style-scope ytd-video-meta-block">조회수 2676만회</span>, <span class="inline-metadata-item style-scope ytd-video-meta-block">3년 전</span>, <span class="inline-metadata-item style-scope ytd-video-meta-block">조회수 1910만회</span>, <span class="inline-metadata-item style-scope ytd-video-meta-block">2년 전</span>, <span class="inline-metadata-item style-scope ytd-video-meta-block">조회수 1287만회</span>, <span class="inline-metadata-item style-scope ytd-video-meta-block">2년 전</span>]


In [15]:
print(view_upload[0]) # 조회수

<span class="inline-metadata-item style-scope ytd-video-meta-block">조회수 2676만회</span>


In [16]:
print(view_upload[1]) # 업로드 시기

<span class="inline-metadata-item style-scope ytd-video-meta-block">3년 전</span>


In [17]:
# 짝수 인덱스는 조회수, 홀수 인덱스는 업로드 시기
print(view_upload[0].text)
print(view_upload[1].text)

조회수 2676만회
3년 전


### 동영상의 정보를 조회수로 가져오는 함수 만들기

In [12]:
# 매개변수를 키워드(컴색할 단어)를 받아서 동영상 정보를 조회순으로
# 가져오는 함수 작성
# 동영상의 제목, 링크, 조회수, 업로드 시기 추출
# 결과 데이터 프레임을 csv로 내본낸다.

def get_data_from_youtube(keyword, is_scroll=False):
    scroll_count = 10 # 스크롤 회수
    base_url = 'https://www.youtube.com/'
    search_word = f'results?search_query={keyword}'
    search_option = '&sp=CAMSAhAB' # 조회수 기준으로 정렬
    # 접속하고자 하는 웹사이트
    url = base_url + search_word + search_option
    driver.get(url) # 웹 브라우저를 실행해서 지정한 url에 접속
    time.sleep(3)

    # 검색어에 따라 거의 무한대의 결과물이 나올 수도 있고,
    # 또는 관련성이 적은 데이터로 노출이 될 수 있어서
    # 특정 횟수(scroll_count)만큼만 스크롤 내린다.
    if is_scroll:
        # 수직(y축 방향)으로 스크롤 동작하기
        y = 0 # y축 방향으로 스크롤 이동할 거리 초기화
        y_step = 1000 # 1000픽셀(조금씩 조금씩 스크롤 내려가는 방식)
        for k in range(scroll_count):
            y += y_step # 반복할 때마다 y축 방향으로 더해지는 거리 지정
            script = f'window.scrollTo(0,{y})' # 자바스크립트(스크롤 해라)
            driver.execute_script(script) # y축 방향으로 스크롤
            time.sleep(3) # 결과를 받아올 때까지 잠시 기다린다.
    
    html = driver.page_source # 접속후에 해당 page의 html 코드를 가져온다.
    soup = bs(html,'html.parser')

    # 1. 동영상 제목과 url 추출
    title_hrefs = soup.select('a#video-title')
    titles = []
    urls = []
    for title_href in title_hrefs:
        title = title_href['title'] # 태그 안에서 title 속성 값 가져오기
        url = base_url + title_href['href'] # href 속성의 값 가져와 기본 url과 합친다.
        titles.append(title)
        urls.append(url)

    # 2. 조회수와 업로드 시기 추출하기(업로드 시기가 없는 경우도 있다.)
    tags = soup.select('ytd-video-renderer.style-scope.ytd-item-section-renderer')
    views = []
    uploads = []  
    for tag in tags:
        view_uploads = tag.select('span.inline-metadata-item.style-scope.ytd-video-meta-block')
        view_num = view_uploads[0] # 인덱스가 짝수인 요소 선택
        view = view_num.get_text() # 조회수 추출
        view = view.split()[1]
        views.append(view)

        # 0번 인덱스 -> 조회수, 1번 인덱스 -> 업로드 시기
        if len(view_uploads) == 2:
            upload = view_uploads[1].get_text() # 업로드 시기 추출
        else:
            upload = ''
        uploads.append(upload)

    # 3. 추출된 정보를 모은다.
    search_results = []
    # 네 개의 리스트를 합쳐서(zip 함수) for문 돌린다.
    for title, url, view, upload in zip(titles, urls, views, uploads):
        search_result = [title, url, view, upload]
        search_results.append(search_result)

    # 4. 추출한 결과를 판다스 데이터프레임 형식으로 변환
    import pandas as pd
    df = pd.DataFrame(search_results, columns = ['제목','링크','조회수','업로드'])

    return df


In [13]:
df = get_data_from_youtube('tensorflow', is_scroll=True)

In [20]:
df.head()

,제목,링크,조회수,업로드
0,Machine Learning for Everybody – Full Course,https://www.youtube.com//watch?v=i_LwzRVP7bg&t...,681만회,2년 전
1,TensorFlow 2.0 전체 과정 - 초보자를 위한 Python 신경망 튜토리얼,https://www.youtube.com//watch?v=tPYj3fFJGjk&p...,312만회,4년 전
2,딥러닝 기초: 소개 및 개요,https://www.youtube.com//watch?v=O5xeyoRL95U&t...,235만회,5년 전
3,"처음부터 신경망 구축(Tensorflow/Pytorch 없음, numpy 및 수학만...",https://www.youtube.com//watch?v=w8yWXqWQYmU&p...,214만회,3년 전
4,여러분의 커뮤니티가 TensorFlow.js로 만들어내는 놀라운 것들 - #Made...,https://www.youtube.com//watch?v=WvHr-yGnGPw&p...,174만회,2년 전


In [21]:
df.to_csv(out_path+'/ytsearch.csv', index = False)
df2 = pd.read_csv(out_path+'/ytsearch.csv')
df2.head()

,제목,링크,조회수,업로드
0,Machine Learning for Everybody – Full Course,https://www.youtube.com//watch?v=i_LwzRVP7bg&t...,681만회,2년 전
1,TensorFlow 2.0 전체 과정 - 초보자를 위한 Python 신경망 튜토리얼,https://www.youtube.com//watch?v=tPYj3fFJGjk&p...,312만회,4년 전
2,딥러닝 기초: 소개 및 개요,https://www.youtube.com//watch?v=O5xeyoRL95U&t...,235만회,5년 전
3,"처음부터 신경망 구축(Tensorflow/Pytorch 없음, numpy 및 수학만...",https://www.youtube.com//watch?v=w8yWXqWQYmU&p...,214만회,3년 전
4,여러분의 커뮤니티가 TensorFlow.js로 만들어내는 놀라운 것들 - #Made...,https://www.youtube.com//watch?v=WvHr-yGnGPw&p...,174만회,2년 전


## 3 유튜브 댓글 수집

In [14]:
# 옵션 설정
options = webdriver.ChromeOptions()

# 드라이버 실행
driver = webdriver.Chrome(options=options)

In [15]:
# 댓글의 댓글은 수집 안 함
# 1. 반응형 디자인이라 화면의 넓이에 따라 html코드가
# 달려져서 브라우저의 폭을 고정
# 2. ajax 방식이라 화면을 아래로 스크롤 해야 한다.

# 첫번째. 스크롤 내리기
# 주소 : https://www.youtube.com/watch?v=avot35M_ETM
url = 'https://www.youtube.com/watch?v=avot35M_ETM'
driver.get(url)

driver.set_window_size(800,800) # 웹 브라우저의 폭을 고정
driver.implicitly_wait(10)
time.sleep(4)

# 맨 처음에 스크롤 조금 내려주기
# send_keys() : 키 입력을 구현
driver.find_element(By.CSS_SELECTOR, 'html').send_keys(Keys.PAGE_DOWN) # page down 키 클릭 구현

# 스크롤 끝까지 내려주기
driver.find_element(By.CSS_SELECTOR,'html').send_keys(Keys.END)
time.sleep(2)

In [24]:
# 옵션 설정
options = webdriver.ChromeOptions()

# 드라이버 실행
driver = webdriver.Chrome(options=options)

# 2) 댓글 스크롤 끝까지 내리기
url = 'https://www.youtube.com/watch?v=avot35M_ETM'
driver.get(url)

driver.set_window_size(800,800) # 웹 브라우저의 폭을 고정
driver.implicitly_wait(10)
time.sleep(4)

# 스크롤이 가장 아래에 온 시점을 확인하기!

height_old = -1 # 스크롤 내리기 전의 페이지 높이 : 초기값 음수로 설정
height_new = -2 # 스크롤 내린 후의 페이지 높이
# 자바 스크립트 코드로 스크롤을 내리고, 전/후 높이가 같으면
# 끝까지 온 것!

# 스크롤을 내려도 이전의 높이와 같으면 종료
while height_old != height_new:
    height_old = height_new # 스크롤 내리기 전에 페이지의 높이를 저장 -> 비교 목적
    height_new = driver.execute_script('return document.querySelector(\'html\').scrollHeight')  # 내려간 스크롤 ->  페이지의 높이를 구함.
    driver.execute_script(f'window.scrollTo(0, {str(height_new)})') # 스크롤이 0부터 height_new까지 내림
    print(height_old,'/',height_new) # 확인 용도로 출력
    time.sleep(5)

-2 / 2879
2879 / 6235
6235 / 8915
8915 / 9489
9489 / 9489


In [16]:
# 3) 댓글 파싱
html = driver.page_source # 스크롤을 끝까지 내린 후에 html 구한다.
soup = bs(html,'html.parser')
# #content-text > span
comments = soup.select('#content-text') # #->id 속성 말한다.
# print(comments)

In [17]:
comments_result = [] # 전체 댓글 저장
for i, comment in enumerate(comments, start = 1):
    # print(f'{i}. {comment.text.strip()}')
    comments_result.append(comment.text)

In [27]:
# 4) 댓글을 csv로 저장
df = pd.DataFrame(comments_result, columns=['댓글'])
df.to_csv(out_path+'/yt_comments.csv', index = False)
df2 = pd.read_csv(out_path+'/yt_comments.csv')
df2.head()

,댓글
0,"와.. 매번 어려워하던 진법변환, 비트연산 개념인데 제가 여지껏 공부한 방법 중에 ..."
1,"흥달쌤이 만약 고등학교 수학선생이었다면, 아마 그 학생들은 모두다 수학을 좋아하고,..."
2,댓글 처음 남겨요. 나만 이해 안되나? 싶었는데 다른 분들 댓글 보니 비슷한 포인트...
3,선생님! 영상 11분에 5를 2진수로 나누면 왜 101이 아니라 0101 인가요? ...
4,ㄷㄷㄷ.. 공부하다 현타와서 보러왔는데 한방에 이해됐습니다 감사합니다


## 4 네이버 이미지 수집

In [18]:
# 옵션 설정
options = webdriver.ChromeOptions()

# 드라이버 실행
driver = webdriver.Chrome(options=options)

In [19]:
# 1) 검색어입력 및 검색어 폴더 생성
keyword = 'lebao'

# 키워드로 폴더 생성
path  = f'./output/naver_image/{keyword}/'
if not os.path.exists(path):
    os.makedirs(path)

# 2) 브라우저 실행 및 페이지를 끝까지 내리기
driver.set_window_size(1200,800)
url = f'https://search.naver.com/search.naver?ssc=tab.image.all&where=image&sm=tab_jum&query={keyword}'
driver.get(url)
time.sleep(3)

# <img src="https://search.pstatic.net/sunny/?src=https%3A%2F%2Fi.pinimg.com%2F736x%2F3e%2F86%2F4e%2F3e864e548c36bbfaebaa5dc9b63e362b.jpg&amp;type=a340" 
# class="_fe_image_tab_content_thumbnail_image" alt="Lebao Fubao Huibao Lebao Fubao Huibao【2024】 | ジャイアントパンダ, パンダ" data-image-viewer-trigger="" 
# data-image-viewer-img-id="image_sas:web_685c43f658ec62214114ee1dcd9d7729" data-thumb-width="340" data-thumb-height="589" style="">

img_tag_selector = 'img._fe_image_tab_content_thumbnail_image' # 이미지 태그 속성
img_tags = driver.find_elements(By.CSS_SELECTOR, img_tag_selector)
print(len(img_tags))
# 스크롤 내릴 때 마다 이미지 출력
cnt = 0
while True:
    if len(img_tags) == cnt:
        break
    # print(f'{cnt+1}번째 작업중 / 총 {len(img_tags)}')
    img_url = img_tags[cnt].get_attribute('src')
    # print(img_url)

    cnt += 1
    if cnt % 30 ==0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 스크롤 내리는 스크립트
        time.sleep(3)
        img_tags = driver.find_elements(By.CSS_SELECTOR,img_tag_selector) # 스크롤 내릴 때마다 새로 수집

time.sleep(5)

# 3) html 소스 파싱
html = driver.page_source
soup = bs(html,'html.parser')
tags = soup.select(img_tag_selector)
# print('-'*50)
# print(len(tags))
# print('-'*50)
# print(tags)
# print('-'*50)


49


In [30]:
# 4) 파싱할 url로 이미지 다운로드
# 크롬 개발자 도구에서 보는 코드와 diver.page_source로 읽은 코드사이에
# 차이가 있을 수 있다.
# --> 출력해보고 작업해야 한다.
print(tags[0].attrs) 

{'src': 'https://search.pstatic.net/sunny/?src=https%3A%2F%2Fi.pinimg.com%2F736x%2Fa7%2Fe4%2F4f%2Fa7e44fba40acb3c97a7b4b711cbb267f.jpg&type=a340', 'class': ['_fe_image_tab_content_thumbnail_image'], 'alt': "Lebao Ghim trên Panda's | Gấu trúc, Gấu, Đang yêu", 'data-image-viewer-trigger': '', 'data-image-viewer-img-id': 'image_sas:web_7dac81a6c4426a5772118d67b20b82da', 'data-thumb-width': '340', 'data-thumb-height': '425', 'onerror': 'naver.common.handleImgError(this)', 'style': ''}


In [31]:
print(tags[15].attrs) 

{'src': 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'class': ['_fe_image_tab_content_thumbnail_image'], 'alt': 'Fiona | .lebao. | Flickr Fiona', 'data-lazysrc': 'https://search.pstatic.net/sunny/?src=https%3A%2F%2Fc1.staticflickr.com%2F7%2F6190%2F6038619260_692dd91e16_b.jpg&type=a340', 'data-img-alias': 'lazyLoaded', 'data-image-viewer-trigger': '', 'data-image-viewer-img-id': 'image_sas:web_bdd6236a6d988622b18dd9c3e9bf4cc3', 'data-thumb-width': '340', 'data-thumb-height': '510', 'onerror': 'naver.common.handleImgError(this)', 'style': 'width: 213px;height: 319.5px', 'data-obstatus': 'ob-once'}


In [20]:
for idx, v in enumerate(tags,start=1):
    img_url = v.attrs['src']
    if img_url[:4] == 'http':
        req.urlretrieve(img_url, f'{path}/{idx}.png') # 이미지 저장 코드
    else:
        img_url = v.attrs['data-lazysrc']
        req.urlretrieve(img_url, f'{path}/{idx}.png')
    # print(f'{idx} / {len(tags)} / {img_url}')